## Properties
A property is a data descriptor that triggers function calls upon access to an attribute. 

Its signature is: `property(fget=None, fset=None, fdel=None, doc=None)`

In [50]:
class C:
    def getx(self): return self.__x
    def setx(self, value): self.__x = value
    def delx(self): del self.__x
    x = property(getx, setx, delx, "I'm the 'x' property.")
c = C()
c.x = 9
c.__dict__   # note how the meant-to-be-private __x becomes _C__x 

{'_C__x': 9}

In [ ]:
The alternative is to use the property decorator:

In [53]:
class C: 
    def __init__(self):  
        self._x = None 
    @property
    def x(self):                # this defines the getter (__get__ method), with the method name as attribute
        "I'm the 'x' property."
        return self._x
    @x.setter                   # property.setter stores its arg (a function) as the __set__ method 
    def x(self, value):   
        self._x = value 
    @x.deleter
    def x(self): 
        del self._x
help(C.x)
C.x

Help on property:

    I'm the 'x' property.



In [56]:
class C: 
    def __init__(self):  
        self._x = None 
    @property
    def x(self):                # this makes x into a constant
        return 42
    #@x.setter                  # by not adding a setter, the attribute becomes read_only 
    #def x(self, value):   
    #    self._x = value 
    @x.deleter
    def x(self): 
        del self._x
c = C()
print(c.x)
c.x = 9

42


AttributeError: can't set attribute

Properties are very useful, for instance to add checks on attribute values. They are an example of a `descriptor` type. A descriptor type is any type/class that provides a `__get__` and/or a `__set__` method (plus optionally a __del__ method). Attribute lookup on an object always checks whether the class of the object stores a descriptor-type object on that attribute, and if so uses that descriptor object to handle getting and/or setting of values. It is up to you where the actual value is stored: this will normally occur on the object concerned, under the name of the attribute or some "private" name, but it can also be stored in some other datastructure kept on or managed by the descriptor object. 

In [ ]:
class Property(object):
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

## Attribute lookup

In [23]:
class Descriptor:   # indirect way of getting __dict__ is just for emulate 
    def __init__(self, name):
        self.name = name
    def __get__(self, obj, type=None):
        print("get value", type)
        return obj.__dict__[self.name] if obj else self
    def __set__(self, obj, value):
        obj.__dict__[self.name] = value
        print("value set")

Attribute lookup is handled by `__getattribute__(self, att)`. The first step is to look for `att` on the class of `self`. If it is found on the class, directly or inherited from its bases, and its value is an overriding descriptor, i.e. an object with a `__set__` method, that descriptor object will be used to get/set the attribute. If no such descriptor is found, the attribute will be searched for on the object itself. If that also fails, the class of the object (and its supers) will again be searched for the att. If the value is a descriptor (this second time a non-overriding descriptor with just a `__get__` will suffice) that descriptor will again be used to get the proper value (which as we will see will probably be a method of some sort), else the value is simply returned. 

In [24]:
class A:
    b = Descriptor('b')
class B(A):
    def b(self): print("B.b")   # descriptor shadowed in B objects 
a = A()
b = B()
a.b = 9                         # value set by descriptor
print (a.b, a.__dict__)         # stored on object a
b.b()                           # calls c method on B (using the __get__ of function stored there, as we will see)
b.b = 9                         # this value will be stored on b and shadow the synonymous method on B
b.__dict__['b']= 3
b.__dict__, b.b
#b()

value set
get value <class '__main__.A'>
9 {'b': 9}
B.b


({'b': 3}, 3)

In [65]:
class A:
    pass
a = A()
a.b = 9
print(a.__dict__, a.b)
A.b = Descriptor('b')
print('now access to b is handled by descriptor: notice message this time!')
print(a.__dict__, a.b)

{'b': 9} 9
now access to b is handled by descriptor: notice message this time!
get value
{'b': 9} 9


Let's try to emulate the (builtin) attribute lookup used by Python. We first need a new Type to handle `class.__getattribute__(att)`.

**NOTE**: You may first want read the [Classes](Classes.ipynb) notebook if you are not familiar with metaclasses. And yes, I know this creates a cycle, as in the classes notebook I give an example that relies on Descriptors. You'll figure it out.

In [65]:
class Type(type):
    ''' Emulate attribute lookup on class'''
    def __getattribute__(cls, att):  # self is a class        
        if att in type.__getattribute__(cls,'__dict__'):
            v = type.__getattribute__(cls,'__dict__')[att]
            if hasattr(v,'__get__'):
                print(f'found __get__ for {att} on {cls}')
                return v.__get__(None, cls)
            else:
                return v
        else:            
            print('checking mro for', att)
            mro = type.__getattribute__(cls,'__mro__')[1:]
            for cl in mro:
                print(cl)
                return Type.__getattribute__(cl,att)
            else:
                raise AttributeError(f'{att} not found ..')

In [78]:
class Descriptor:   # need to adapt to prevent __dict__ lookup to use my emulated __getattribute__ below,
                    # which does not know how to handle dunders (it only handles user attributes in __dict__)
    def __init__(self, name):
        self.name = name
    def __get__(self, obj, type=None):
        print("get value", type)
        return object.__getattribute__(obj,'__dict__')[self.name] if obj else self
    def __set__(self, obj, value):
        object.__getattribute__(obj,'__dict__')[self.name] = value
        print("value set")

In [79]:
class Foo(metaclass=Type):
    @staticmethod
    def c(): print('c ...')
    def b(self): print('b ...')
    a = Descriptor('a')
    pass
Foo.c()
Foo.a

found __get__ for c on <class '__main__.Foo'>
c ...
found __get__ for a on <class '__main__.Foo'>
get value <class '__main__.Foo'>


In [80]:
class Object(metaclass=Type):
    ''' Emulate attribute lookup on class'''
    def __getattribute__(self, att): 
        dict = type.__getattribute__(type(self),'__dict__')
        if att in dict and hasattr(dict[att], '__set__'):
            print(f'1 found __set__ for {att} on {type(self)}')
            return dict[att].__get__(self, None)
        elif att in object.__getattribute__(self,'__dict__'):
            print(f'2 looking for {att} on {self}')
            return self.__dict__[att]
        else:
            v = Type.__getattribute__(type(self),att)
            print(f'3 looking for __get__ on {att} of {type(self)}')
            if hasattr(v,'__get__'):
                print(f'4 found __get__ for {att} on {type(self)}: v{v}')
                return v.__get__(self, type(self))
            else:
                return v          

In [93]:
class Foo(Object):
    @classmethod
    def c(cls): 
        print('c ...', cls)
    def b(self): print('b ...', self)
    a = Descriptor('a')
    pass

Foo.a
f = Foo()
f.b()
f.c()
f.a = 3
f.a

found __get__ for a on <class '__main__.Foo'>
get value <class '__main__.Foo'>
found __get__ for b on <class '__main__.Foo'>
3 looking for __get__ on b of <class '__main__.Foo'>
4 found __get__ for b on <class '__main__.Foo'>: v<function Foo.b at 0x000001707E004DC8>
b ... <__main__.Foo object at 0x000001707E0014C8>
found __get__ for c on <class '__main__.Foo'>
3 looking for __get__ on c of <class '__main__.Foo'>
4 found __get__ for c on <class '__main__.Foo'>: v<bound method Foo.c of <class '__main__.Foo'>>
c ... <class '__main__.Foo'>
value set
1 found __set__ for a on <class '__main__.Foo'>
get value None


3

In [90]:
def foo(): print("fdsfds")
fm = staticmethod(foo)
fm.__get__(object(), type)


<function __main__.foo()>